In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Cropping2D, ZeroPadding2D

import numpy as np
import matplotlib.pyplot as plt

import os
import tifffile
from PIL import Image

import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models.segmentation as models


#The dataset


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


##Image Preprocessing

In [36]:
from torchvision import transforms
import torch

def preprocess_image(image, label):
    # Normalize image by converting it to the range [0, 1]
    image = image / 255.0  # the images are 12-channel, so keep all channels

    # Resize the image and label
    transform_image = transforms.Compose([
        transforms.ToTensor(),  # Convert to Tensor
        transforms.Resize((128, 128))  # Resize to 128x128 if needed
    ])

    transform_label = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((128, 128))  # Resize to 128x128 if needed
    ])

    # Apply transformations
    image = transform_image(image).float()
    label = transform_label(label).long().squeeze(0)  # Convert label to long tensor

    return image, label



In [37]:
class SegmentationDataset(Dataset):
    def __init__(self, image_files, label_files, transform=None):
        self.image_files = image_files
        self.label_files = label_files
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load the image and label
        image = tifffile.imread(self.image_files[idx])  # Load 12-channel image
        label = Image.open(self.label_files[idx])  # Load label as grayscale
        label = np.array(label)  # Convert label to numpy array

        if self.transform:
            image, label = self.transform(image, label)

        return image, label

##Loading the data and creating dataLoader

In [38]:
# Define paths
images_path = '/content/drive/MyDrive/data/images'
labels_path = '/content/drive/MyDrive/data/labels'

# Load all image and label files
image_files = [os.path.join(images_path, f) for f in os.listdir(images_path) if f.endswith('.tif')]
label_files = [os.path.join(labels_path, f) for f in os.listdir(labels_path) if f.endswith('.png')]

# Sort files to ensure they match
image_files.sort()
label_files.sort()

# Create the dataset and DataLoader
train_dataset = SegmentationDataset(image_files, label_files, transform=preprocess_image)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#Loading DeepLabV3

In [42]:
import torch.nn as nn

# Loading the pretrained DeepLabV3+ model
deeplab = models.deeplabv3_resnet101(pretrained=True)

# Modifying the first convolutional layer to accept 12 channels instead of 3
deeplab.backbone.conv1 = nn.Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Set the model to evaluation mode
deeplab.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [46]:
# Assuming image_files and label_files contain validation data as well
val_dataset = SegmentationDataset(image_files, label_files, transform=preprocess_image)

# Create a DataLoader for validation (batch size can be the same as for training or different)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)

#Accuracy

In [47]:
def compute_pixel_accuracy(predictions, labels):
    """
    Compute pixel accuracy (the ratio of correctly predicted pixels).

    Args:
        predictions: Predicted segmentation masks (batch_size, H, W)
        labels: Ground truth segmentation masks (batch_size, H, W)

    Returns:
        accuracy: Pixel accuracy
    """
    correct = (predictions == labels).sum().float()
    total = labels.numel()

    accuracy = correct / total
    return accuracy.item()


In [48]:
num_classes = 2

total_accuracy = 0
num_batches = 0

# Loop through the DataLoader for evaluation
for images, labels in val_loader:  # Replace val_loader with your validation/test DataLoader
    with torch.no_grad():
        images = images.float()  # Ensure images are in float32 format

        # Perform the forward pass to get the predictions
        outputs = deeplab(images)['out']
        predictions = torch.argmax(outputs, dim=1)  # Predicted segmentation map

        # Compute metrics for each batch
        accuracy = compute_pixel_accuracy(predictions, labels)

        # Aggregate results
        total_accuracy += accuracy
        num_batches += 1

# Average results across batches
mean_accuracy = total_accuracy / num_batches

# Display the results
print(f'Mean Pixel Accuracy: {mean_accuracy}')


Mean Pixel Accuracy: 0.9519857431386972
